In [1]:
import os
import sys
sys.path.append("../")
import random

import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.models import Model

import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder
from layer.bayesian_dropout_layer import BayesianDropoutLayer
from collections import Counter


import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

2022-04-03 11:14:50.741857: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-03 11:14:50.775501: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-03 11:14:50.775718: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
train_folder = "/home/fsantiago/projects/medium/datasets/cifar-10/train_aug"
train_labels = "/home/fsantiago/projects/medium/datasets/cifar-10/labels_aug.csv"

In [3]:
labels_df = pd.read_csv(train_labels, index_col="id")
one_hot = OneHotEncoder().fit(labels_df[["label"]])
one_hot_labels = OneHotEncoder().fit_transform(labels_df[["label"]]).toarray()

In [4]:
def get_y(file):
    file_idx = int(file.replace(".png", "")) - 1

    return one_hot_labels[file_idx]

In [5]:
files = os.listdir(train_folder)
files_batch = files.copy()

In [6]:
len(files)

149695

In [7]:
train_size = 120000
validation_size = 10000
test_size = 19695

In [8]:
test_files = [f"{val_file}.png" for  val_file in list(range(1, validation_size))]
val_files = [f"{val_file}.png" for  val_file in list(range(validation_size+1, test_size))]
train_files = [f"{val_file}.png" for  val_file in list(range(test_size+1, train_size))]


In [9]:
train_X = [plt.imread(f"{train_folder}/{file}") for file in train_files]
train_y = [get_y(file) for file in train_files]

validation_X = [plt.imread(f"{train_folder}/{file}") for file in val_files]
validation_y = [get_y(file) for file in val_files]

test_X = [plt.imread(f"{train_folder}/{file}") for file in test_files]
test_y = [get_y(file) for file in test_files]

train_mean = np.array([train.mean() for train in train_X]).mean()

train_X = [img[...,:, :3] - train_mean for img in train_X]
validation_X = [img[...,:, :3] - train_mean for img in validation_X]
test_X = [img[...,:, :3] - train_mean for img in test_X]

In [ ]:
batch_size = 32

def load_batches_img_folder(folder_path, train_size, batch_size):

    files = os.listdir(folder_path)[:train_size]
    files_batch = files.copy()
    batches = []

    while len(files_batch) > 0:
        if len(files_batch) < batch_size:
            batch = files_batch.copy()
        else:
            batch = random.sample(files_batch, batch_size)

        files_batch = list(filter(lambda x: x not in batch, files_batch))
        batches.append(batch)
    
    return batches


def load_batches(folder_path, train_size, batch_size, train_mean):
    file_batches = load_batches_img_folder(folder_path, train_size, batch_size)
    batches = []

    for batch_files in file_batches:
        batch = []
        y = []
        for file in batch_files:
            img = plt.imread(f"{folder_path}/{file}", format="png") - train_mean
            y.append(get_y(file))
            if img.shape[2] != 3:
                img = img[...,:, :3]

            batch.append(img)
        
        batches.append((np.array(batch), y))

    return batches

In [ ]:
batches = load_batches(train_folder, train_size, batch_size, train_mean)

In [ ]:
auto_input = layers.Input(shape=(32, 32, 3))
layer_1 = BayesianDropoutLayer(units=1024, dropout=0.2, l=1e-3, activation=tf.nn.relu)
layer_2 = BayesianDropoutLayer(units=1024, dropout=0.2, l=1e-3, activation=tf.nn.relu)
layer_3 = BayesianDropoutLayer(units=250, dropout=0.2, l=1e-3, activation=tf.nn.relu)

def get_conv_layer(filters, x, dropout=0.25, kernel_size=(3, 3)):

    x = layers.Conv2D(filters, kernel_size=kernel_size, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Dropout(dropout)(x)
    return x

def get_conv_layer_2(filters, x, number_of_layers):

    for i in range(number_of_layers - 1):
        x = layers.Conv2D(filters, kernel_size=(3,3), padding="same")(x)

    x = layers.Conv2D(filters, kernel_size=(3,3), strides=(2, 2), padding="same")(x)
    return x


def get_mini_vgg_19():

    x = get_conv_layer(64, auto_input, kernel_size=(3, 3))
    x = get_conv_layer(64, x, kernel_size=(3, 3))
    x = layers.MaxPooling2D((2, 2), padding="same")(x)

    x = get_conv_layer(128, x)
    x = get_conv_layer(128, x)
    x = layers.MaxPooling2D((2, 2), padding="same")(x)

    x = get_conv_layer(256, x)
    x = get_conv_layer(256, x)
    x = layers.MaxPooling2D((2, 2), padding="same")(x)

    x = get_conv_layer(512, x)
    x = get_conv_layer(512, x)
    x = layers.MaxPooling2D((2, 2), padding="same")(x)

    x = layers.Flatten()(x)

    x = layer_1(x)
    x = layer_2(x)
    x = layer_3(x)

    # x = layers.Dense(512, activation='relu')(x)
    # x = layers.Dropout(0.5)(x)
    # x = layers.Dense(512, activation='relu')(x)
    # x = layers.Dropout(0.5)(x)
    # x = layers.Dense(128, activation='relu')(x)
    # x = layers.Dropout(0.5)(x)

    x = layers.Dense(10, activation='softmax')(x)

    return x


def get_mini_plain():

    x = get_conv_layer(64, auto_input, 3)
    x = get_conv_layer(128, x, 4)
    x = get_conv_layer(256, x, 6)
    x = get_conv_layer(512, x, 2)

    x = layers.AveragePooling2D((2, 2), padding="same")(x)

    x = layers.Flatten()(x)

    x = layers.Dense(1000, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Dense(10, activation='softmax')(x)

    return x

x = get_mini_vgg_19()
model = Model(auto_input, x)
model.summary()


2022-04-03 11:08:47.983488: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-03 11:08:47.984305: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-03 11:08:47.984634: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-03 11:08:47.984824: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 64)        1792      
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 32, 32, 64)        0         
                                                                 
 dropout (Dropout)           (None, 32, 32, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        36928     
                                                             

2022-04-03 11:08:48.688337: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [ ]:
optimizer = tf.keras.optimizers.Adam()


def compute_loss(labels, logits):
    loss = tf.keras.losses.categorical_crossentropy(labels, logits)

    regularization =  (layer_1.regularization + layer_2.regularization + layer_3.regularization)
    loss = loss + regularization

    return loss, regularization


@tf.function
def train_step(x, y): 
    # Use tf.GradientTape()
    with tf.GradientTape() as tape:

        y_hat = model(x)      
        loss, regularization = compute_loss(y, y_hat)
        
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(grads_and_vars=zip(grads, model.trainable_variables))
               
        return loss, regularization

In [ ]:
import time

epochs = 40
_batches = batches
print(len(_batches))

for epoch in range(epochs):
    start_time = time.time()

    for batch in _batches:
        x_batch, y_batch = batch
        x_batch = np.array(x_batch)
        loss, regularization = train_step(x_batch, y_batch)

    end_time = time.time()

    print(f"{epoch} of {epochs} => time:{end_time - start_time} loss:{np.array(loss).mean()}, reg:{np.array(regularization).mean()}")

    # if epoch % 10 == 0:
        
    #     val_predictions = model.predict(np.array(validation_X))
    #     val_predictions = list(map(lambda x: x[0], one_hot.inverse_transform(val_predictions)))
    #     val_labels = list(map(lambda x: x[0], one_hot.inverse_transform(validation_y)))
    #     val_acc = accuracy_score(val_labels, val_predictions)

    #     print(f"\n Val acc: {val_acc} \n")

469


2022-04-03 11:08:50.777495: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8300
2022-04-03 11:08:51.131432: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-04-03 11:08:52.033026: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.34GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-04-03 11:08:52.146867: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.14GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-04-03 11:08:52.612357: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.1

0 of 40 => time:268.97793316841125 loss:2.303684949874878, reg:0.00031900580506771803


KeyboardInterrupt: 

In [ ]:
val_predictions = model.predict(np.array(validation_X))
val_predictions = list(map(lambda x: x[0], one_hot.inverse_transform(val_predictions)))
val_labels = list(map(lambda x: x[0], one_hot.inverse_transform(validation_y)))


test_predictions = model.predict(np.array(test_X))
test_predictions = list(map(lambda x: x[0], one_hot.inverse_transform(test_predictions)))
test_labels = list(map(lambda x: x[0], one_hot.inverse_transform(test_y)))

In [ ]:
accuracy_score(test_labels, test_predictions)

0.0999099909990999

In [ ]:
df = pd.DataFrame({"label": test_labels, "predictions": test_predictions})
df["label"].unique()

array(['frog', 'truck', 'deer', 'automobile', 'bird', 'horse', 'ship',
       'cat', 'dog', 'airplane'], dtype=object)

In [ ]:
labels = ['bird', 'cat', 'ship', 'horse', 'dog', 'deer', 'airplane', 'automobile', 'frog', 'truck']


for label in labels:
    df_label = df[df["label"] == label].copy()
    acc = accuracy_score(df_label["label"], df_label["predictions"])
    print(f"{label}   {acc}")

bird   0.0
cat   0.0
ship   0.0
horse   0.0
dog   0.0
deer   1.0
airplane   0.0
automobile   0.0
frog   0.0
truck   0.0


In [ ]:
df_dog = df[df["label"] == 'bird'].copy()

df_dog.groupby(['predictions']).count()

,label
predictions,
deer,1032


In [ ]:
accuracy_score(val_labels, val_predictions)

0.10088714668867341

In [ ]:
model.save_weights("./normal_nn_weigths/weights.ckpt")

In [ ]:
test_folder = "/home/fsantiago/projects/medium/datasets/cifar-10/test/test/"
model.load_weights("./bayesian_weigths/weights.ckpt")